In [22]:
from pysat.solvers import Glucose3
class Coloring:
    def __init__(self) -> None:
        self.solver = Glucose3()
        self.row = 0
        self.col = 0
        self.maze = []

    # đọc file
    def readFile(self, fileName):
        with open(fileName, 'r') as f:
            lines = f.readlines()
            self.row, self.col = map(int, lines[0].strip().split())
            for line in lines[1:]:
                row_data = []
                for item in line.split():
                    if item.isdigit():
                        row_data.append(int(item))
                    else:
                        row_data.append(item)
                self.maze.append(row_data)
        f.close()

    # tô màu
    def red(self,text):
        return f"\033[1;37;41m{text}\033[0;0m"
    def green(self,text):
        return f"\033[1;37;42m{text}\033[0;0m" 
    
    def solve(self):
        

        

        # in kết quả
        if self.solver.solve():
            number = 0
            model = self.solver.get_model()
            for i in range(self.row):
                for j in range(self.col):
                    if model[number] > 0:
                        print(self.green(self.maze[i][j]), end = '')
                    else:
                        print(self.red(self.maze[i][j]), end = '')
                    number += 1
                print()
        else:
            print("UNSAT")


# test
c = Coloring()
c.readFile("example1.txt")
c.solve()


IndexError: list index out of range